<a href="https://colab.research.google.com/github/ViktorSivek/Mistral_voicebot/blob/main/Kopie_se%C5%A1itu_mistral_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech to text Whisper

In [1]:
# Enable third-party widgets in Google Colab
from google.colab import output
output.enable_custom_widget_manager()

# Install
!pip install ipywebrtc
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

# Import necessary classes
from IPython.display import Audio
from ipywebrtc import AudioRecorder, CameraStream
import re
import subprocess
import io
import sys

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 3.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-u5kqfjqx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-u5kqfjqx
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-man

In [2]:
# Set up an attempt to record audio
audio_stream = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=audio_stream)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [3]:
# Save the recorded audio to a file
audio_data = recorder.audio.value
with open('output.mp3', 'wb') as f:
    f.write(audio_data)

In [18]:
# Run Whisper and capture the output
try:
    result = subprocess.run(
        ['whisper', 'output.mp3', '--model', 'medium.en'],
        capture_output=True, text=True, check=True
    )
    whisper_output = result.stdout
except subprocess.CalledProcessError as e:
    print("Failed to transcribe audio:", e)
    whisper_output = ""

# Extract only the text from the Whisper output
if whisper_output:
    clean_text = re.sub(r"^\[.*?\]\s{2}", "", whisper_output, flags=re.MULTILINE)
    final_query = "Answer only from provided file. " + clean_text
    print(final_query)
else:
    print("No output from Whisper.")

Answer only from provided file. I cut myself with a knife and it's bleeding, what I should do.



# Mistral RAG


In [5]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install faiss-gpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from ipywidgets import Dropdown, Output, Button
from IPython.display import display
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
from ipywidgets import Dropdown, Output, Button
from IPython.display import display
import os

# Dropdown for selecting help type
dropdown = Dropdown(
    options=[('First Aid', 'first_aid'), ('Mental Health', 'mental_health')],
    value='first_aid',
    description='Help Type:',
    disabled=False,
)

# Output area for displaying results or messages
output = Output()

# Button to start the process
button = Button(description="Load Data")

display(dropdown, button, output)

Dropdown(description='Help Type:', options=(('First Aid', 'first_aid'), ('Mental Health', 'mental_health')), v…

Button(description='Load Data', style=ButtonStyle())

Output()

In [16]:
# Define the action for the button click
def load_data(b):
    with output:
        output.clear_output()
        help_type = dropdown.value
        folder_path = "/content/drive/MyDrive/data/" if help_type == 'first_aid' else "/content/drive/MyDrive/mental/"

        if not os.path.exists(folder_path):
            print("Folder does not exist. Please check the path.")
            return

        # Load PDF files
        loader = PyPDFDirectoryLoader(folder_path)
        global data
        data = loader.load()
        print(f"Data loaded from {folder_path}")

button.on_click(load_data)

In [17]:
# Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

# Download the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

# Import Model
llm = LlamaCpp(
    streaming=True,
    model_path="/content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

print("Model and embeddings are ready.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

Model and embeddings are ready.


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'mistralai_mistral-7b-instruct-v0.1', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_token_id': '1', 'llama.attention.head_count': '32', 'llama.block_count': '32', 'llama.attention.head_count_kv': '8', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '15'}
Using fallback chat format: None


In [19]:
# Setup QA model
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

# Define a function to run queries
def run_query(query):
    return qa.invoke(query)

# Example usage
query_result = run_query(final_query)
print(query_result)


llama_print_timings:        load time =    3267.72 ms
llama_print_timings:      sample time =     100.53 ms /   162 runs   (    0.62 ms per token,  1611.54 tokens per second)
llama_print_timings: prompt eval time =  665428.43 ms /  1339 tokens (  496.96 ms per token,     2.01 tokens per second)
llama_print_timings:        eval time =  119345.05 ms /   161 runs   (  741.27 ms per token,     1.35 tokens per second)
llama_print_timings:       total time =  786000.06 ms /  1500 tokens


{'query': "Answer only from provided file. I cut myself with a knife and it's bleeding, what I should do.\n", 'result': " From the provided file, it is stated that bleeding can be severe or continued and if not controlled, it can be potentially fatal. If there are no foreign bodies or protruding bones in the wound, direct pressure should be applied to stop the bleeding as quickly as possible (refer to BLEEDING section). After stopping the bleeding, the wound should be dressed with a sterile dressing and pad, and then bandaged firmly. If blood seeps through the bandage, it should not be removed, but replaced with a new pad. It is important to monitor the casualty for shock and seek medical aid urgently (refer to BLEEDING section).\n\nIf you have any other question about this topic or if you need more information, please don't hesitate to ask."}


# Text to speech Bark

In [20]:
!pip install git+https://github.com/suno-ai/bark.git
!pip install nltk

from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import json
import re
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np

# for ram problem
# import os
# os.environ["SUNO_OFFLOAD_CPU"] = "True"
# os.environ["SUNO_USE_SMALL_MODELS"] = "True"

# download and load all models
preload_models()

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-na84rym_
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-na84rym_
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567

text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 136MB/s]


In [35]:
# Assuming query_result is already a dictionary
if isinstance(query_result, dict):
    # Directly access the 'result' key from the dictionary
    result_text = query_result.get('result', 'No result found')

    # Optionally, use or display the extracted result
    print(result_text)
else:
    print("The query result is not a dictionary")

 From the provided file, it is stated that bleeding can be severe or continued and if not controlled, it can be potentially fatal. If there are no foreign bodies or protruding bones in the wound, direct pressure should be applied to stop the bleeding as quickly as possible (refer to BLEEDING section). After stopping the bleeding, the wound should be dressed with a sterile dressing and pad, and then bandaged firmly. If blood seeps through the bandage, it should not be removed, but replaced with a new pad. It is important to monitor the casualty for shock and seek medical aid urgently (refer to BLEEDING section).

If you have any other question about this topic or if you need more information, please don't hesitate to ask.


In [36]:
# Tokenize the text into sentences
nltk.download('punkt')
sentences = sent_tokenize(result_text)

print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[' From the provided file, it is stated that bleeding can be severe or continued and if not controlled, it can be potentially fatal.', 'If there are no foreign bodies or protruding bones in the wound, direct pressure should be applied to stop the bleeding as quickly as possible (refer to BLEEDING section).', 'After stopping the bleeding, the wound should be dressed with a sterile dressing and pad, and then bandaged firmly.', 'If blood seeps through the bandage, it should not be removed, but replaced with a new pad.', 'It is important to monitor the casualty for shock and seek medical aid urgently (refer to BLEEDING section).', "If you have any other question about this topic or if you need more information, please don't hesitate to ask."]


In [37]:
# Generate audio for each sentence
all_audio_arrays = []

for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt="v2/en_speaker_1")
    all_audio_arrays.append(audio_array)

100%|██████████| 22/22 [00:21<00:00,  1.03it/s]


In [38]:
import numpy as np

# Assuming all audio clips are of the same sample rate
def concatenate_audios(audio_clips):
    return np.concatenate(audio_clips)

# Concatenate all audio arrays
final_audio_array = concatenate_audios(all_audio_arrays)

# Save or play the final audio
from IPython.display import Audio

# Replace SAMPLE_RATE with your actual audio sample rate
Audio(final_audio_array, rate=SAMPLE_RATE)